TLDR dijkstra's finds the shortest path from a given node to EVERY other node. orr maybe stops at the intended end node ig

to init,
1. import all the nodes and distances between nodes DONE
2. place everything into 1 data structure DONE
to start traversing the graph,
3. init ?list? with dist. from start node of all nodes. SN-SN is 0, SN-everything else is infinite
4. init "path" ?hash map? to retrace path.
5. create priority queue (min-heap) based on shortest dist. from SN. but how????? using adjacency matrix.
- get directly adjacent to start node. check if in 'not added' list, then remove.
- get directly adjacent to adjacent. check if in 'not added' list, then remove.
- repeat until 'not added' is empty, or 
repeat:
6. pop node NN at top of priority queue.
7. for each neighbour of NN, check if path through NN offers a shorter path to neighbour. (a lot of the rewriting will start happening, since literally anything will be shorter than infinity.)
until EITHER end node is reached OR all nodes are reached


*** should I be using a linked list for this?
*** does this mean that dijkstra's algo only checks each node once? Yes.. I think so?

In [50]:
import pandas as pd
from heapq import heapify, heappop, heappush

In [3]:
#load data
intervals=pd.read_csv('intervals.csv')
stn_name=pd.read_csv('station_name.csv')

In [4]:
#preprocess
intervals=intervals.map(lambda x: x.strip() if isinstance(x, str) else x)
stn_name=stn_name.map(lambda x: x.strip() if isinstance(x, str) else x)

In [5]:
#save dataframe as csv
intervals.to_csv('intervals.csv', index=False)
stn_name.to_csv('station_name.csv', index=False)

In [6]:
# intervals.head(10)

In [7]:
# stn_name.head(10)

In [ ]:
class Graph:
	def __init__(self):
		self.neighbours_list = {}
		self.no_of_nodes = 0
		self.vertex_data = {}

	# add rmv NODES/VERTICES
	def add_vertex_data(self, vertex, data): #for vertex. node.
		self.vertex_data[vertex] = data
		self.neighbours_list[vertex]={}
		self.no_of_nodes+=1

	def remove_vertex_data(self, vertex):
		#call remove edge first, then remove entire vertex from vertex_data and neighbours_list
		#count remove 1
		pass

	# add rmv CONNECTIONS/EDGES
	def add_edge(self, start, end, time):
		if start in self.vertex_data and end in self.vertex_data:
			self.neighbours_list[start][end] = time

	def remove_edge(self, start, end):
		#call remove edge for all edges connected to this vertex: first remove on this side, then use the key to remove on the other side
		pass

	# getters
	def get_all_station_code(self, station_name):
		#preprocess
		station_name = station_name.lower().strip()
		codes = [k for k, v in self.vertex_data.items() if v.lower() == station_name]
		return codes # CURRENTLY ONLY RETURNS THE FIRST !!!!!!!!
	
	def get_station_code(self, station_name):
		return self.get_all_station_code(station_name)[0]
	
	def get_station_name(self, station_code):
		name = self.vertex_data[station_code]
		return name
	
	# helper functions for dijkstra
	def shortest_time(self, source_stn):
		# init time list
		time_to={node:float('inf') for node in self.vertex_data.keys()}
		time_to[source_stn] = 0

		# init priority queue
		pq = [(0,source_stn)] #1. start with start node
		heapify(pq)

		# record visited nodes
		visited = []
		path =[source_stn]

		#iterate through list
		while pq: #pq is not empty (aka there's reachable nodes you haven't calculated dist. for yet.)
			curr_time, curr_node = heappop(pq)
			if curr_node in visited: #3. if already visited, remove without doing anything.
				continue
			visited.append(curr_node)

			for neighbour, timedist in self.neighbours_list[curr_node].items(): #2. append all neighbours onto PQ; for subsequent runs, append neighbour's neighbours onto PQ
				temp = curr_time + timedist
				if temp < time_to[neighbour]:
					time_to[neighbour] = temp #replace

					heappush(pq, (temp, neighbour))
		
		return time_to
	
	def retrace(self, start, end, time, time_to):
		prev = []
		#A 1 B 2 C 1 D 2 E 1 F
		#init running counter for to_end (confirmed nodes.)
		#if time_to[neighbour]==timedist:
		# then return [neighbour]
		#if (time)=dist_from_neighbour_to_curr + time_to[neighbour],
		#then return [neighbour] + [self.retrace(self, neighbour, end, time-timedist, time_to)]
		
		#how to update time?
		neighbour_list=self.neighbours_list[end].items()

		#decrementing end
		if (start==end):
			return []
		else:
			for neighbour, timedist in neighbour_list:
				if time_to[neighbour]+timedist==time_to[end]:
					recursion_list=self.retrace(start, neighbour,time-timedist,time_to)
					recursion_list.append(neighbour)
					return recursion_list

	
	# RETURNNNNNN
	def path(self, start=None, end=None):
		if start==None:
			start=input("Begin from: ")
		if end==None:
			end=input("End at: ")
		
		start_code=self.get_station_code(start)
		end_code=self.get_station_code(end)

		time_to=self.shortest_time(start_code)
		result=time_to[end_code]
		path=self.retrace(start_code, end_code, result, time_to)

		print(f"Shortest time from {start} ({start_code}) to {end} ({end_code}) is {result} minutes.\nPath: {path}")
		return result, path
	
	def flatten_path(self, path):
		#compare curr and next, flatten timings and paths on the same line
		#specify interchanges
		pass

	def check_interchange(self,stn_code):
		name=self.get_station_name(stn_code)
		if len(self.get_all_station_code(name))>1:
			return True
		else:
			return False

	def check_same_line(self, code1, code2):
		return (code1[0:2]==code2[0:2])
		
	
	def __str__(self):
		result = "List of nodes:\n"
		for key,value in self.vertex_data.items():
			result += f"{key}: {value}\n"
		return result

In [163]:
a=[1,2,3,4]
print(a[0:1])

[1]


In [161]:
size=len(stn_name)
interval_size=len(intervals)
mrt_map=Graph()

for i in range(size):
    mrt_map.add_vertex_data(stn_name.iloc[i,0],stn_name.iloc[i,1]) #iloc is used to access the data in the dataframe
    
for i in range(interval_size):
    mrt_map.add_edge(intervals.iloc[i,0], intervals.iloc[i,1], int(intervals.iloc[i,2]))

In [162]:
mrt_map.path('telok blangah','great world')

Shortest time from telok blangah (CC28) to great world (TE15) is 14 minutes.
Path: ['CC28', 'CC29', 'NE1', 'NE3', 'TE17', 'TE16']


(14, ['CC28', 'CC29', 'NE1', 'NE3', 'TE17', 'TE16'])

In [51]:
lst=[(0,2),(9,8)]
print(2 in lst)
print(9 in lst)
print(4 in lst)

False
False
False


In [9]:
for key, value in mrt_map.adj_matrix.items():
	print(f"{key}: {value}")

CC29: {'CC28': 2, 'NE1': 2}
NE1: {'NE3': 4, 'CC29': 2}
NE3: {'NE4': 2, 'NE1': 4, 'EW16': 2, 'TE17': 2}
EW16: {'EW17': 2, 'EW15': 2, 'NE3': 2, 'TE17': 2}
TE17: {'TE18': 2, 'TE16': 2, 'EW16': 2, 'NE3': 2}
NE4: {'NE3': 2, 'DT19': 2}
DT19: {'DT20': 3, 'DT18': 1, 'NE4': 2}
NS24: {'NS25': 3, 'NS23': 2, 'NE6': 2, 'CC1': 2}
NE6: {'NE7': 1, 'NS24': 2, 'CC1': 2}
CC1: {'CC2': 2, 'NS24': 2, 'NE6': 2}
NE7: {'NE8': 1, 'NE6': 1, 'DT12': 2}
DT12: {'DT13': 2, 'DT11': 2, 'NE7': 2}
NE8: {'NE9': 2, 'NE7': 1}
NE9: {'NE10': 3, 'NE8': 2}
NE10: {'NE11': 1, 'NE9': 3}
NE11: {'NE12': 2, 'NE10': 1}
NE12: {'NE13': 3, 'NE11': 2, 'CC13': 2}
CC13: {'CC14': 2, 'CC12': 2, 'NE12': 2}
NE13: {'NE14': 2, 'NE12': 3}
NE14: {'NE15': 2, 'NE13': 2}
NE15: {'NE16': 2, 'NE14': 2}
NE16: {'NE17': 3, 'NE15': 2, 'STC': 2}
STC: {'SW1': 2, 'SE1': 2, 'SW8': 3, 'SE5': 3, 'NE16': 2}
NE17: {'NE18': 2, 'NE16': 3, 'PTC': 2}
PTC: {'PW1': 2, 'PE1': 3, 'PE7': 2, 'NE17': 2}
NE18: {'NE17': 2}
PE1: {'PE2': 1, 'PTC': 3}
PE2: {'PE3': 1, 'PE1': 1}
PE3